## 【コロモゴロフスミルノフ検定（KS検定）】

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [2]:
import glob
win_path = f'../season1_features/4_winner/*.gz'
#  Data Load
base = utils.read_df_pkl('../input/base*')
win_path_list = glob.glob(win_path)
# tmp_path_listには検証中のfeatureを入れてある
# tmp_path_list = glob.glob('../features/5_tmp/*.gz')
# tmp_path_list = glob.glob('../season1_features/5_tmp/*.gz')
tmp_path_list = []
win_path_list += tmp_path_list

train_path_list = []
test_path_list = []
for path in win_path_list:
    if path.count('train'):
        train_path_list.append(path)
    elif path.count('test'):
        test_path_list.append(path)

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
train_feature_list = utils.parallel_load_data(path_list=train_path_list)
test_feature_list = utils.parallel_load_data(path_list=test_path_list)
train = pd.concat(train_feature_list, axis=1)
train = pd.concat([base_train, train], axis=1)
test = pd.concat(test_feature_list, axis=1)
test = pd.concat([base_test, test], axis=1)

100%|██████████| 3/3 [00:00<00:00, 74.04it/s]


In [12]:
# from scipy.stats import ks_2samp
# list_p_value =[]

# for col in tqdm(train.columns):
#     list_p_value.append(ks_2samp(test[col] , train[col])[1])

se_p = pd.Series(list_p_value, index = train.columns).sort_values() 
# list_discarded = list(se_p[se_p < .1].index)
list_discarded = list(se_p[se_p < .3].index)
list_discarded = list(set( list_discarded))
se_p[se_p < .3]

target                                      0.000000
114_mer_merchant_id_M_ID_cd2c0b07e9@        0.036328
110_ker_hist_category_2_1_0_mean            0.041398
152_new_new_purchase_amount_mean@           0.043279
117_ah1_auth1_hist_category_2_mean_mean@    0.049873
152_ah1_auth1_hist_purchase_amount_sum@     0.056719
118_ah0_auth0_hist_category_2_mean_mean@    0.072399
110_ker_auth_category_2_3_0_mean            0.100639
111_ker_new_hist_month_lag_mean@            0.112871
152_new_new_purchase_amount_sum@            0.119271
152_ah1_auth1_hist_purchase_amount_min@     0.122635
111_ker_hist_weekend_mean@                  0.156382
139_com_auth1_continuous_month_max@         0.162315
130_ddf_hist_last10_date_diff_mean@         0.166421
152_ah1_auth1_hist_purchase_amount_max@     0.171231
110_ker_new_category_2_3_0_mean             0.177672
110_ker_auth_category_2_1_0_mean            0.188194
111_ker_hist_month_lag_mean@                0.189343
111_ker_new_hist_category_3_mean_mean@      0.

In [9]:
list_discarded

['118_ah0_auth0_hist_category_2_mean_mean@',
 'target',
 '152_new_new_purchase_amount_mean@',
 '152_ah1_auth1_hist_purchase_amount_sum@',
 '114_mer_merchant_id_M_ID_cd2c0b07e9@',
 '110_ker_hist_category_2_1_0_mean',
 '117_ah1_auth1_hist_category_2_mean_mean@']

In [13]:
import shutil
import re
import os

feature_season = 'season1_features'

list_discarded = list(set( list_discarded))
for feat in list_discarded:
    if feat.count('target'):continue
    train_path = f'{feat[:8]}train_{feat[8:]}.gz'.replace('2_1_0', '2_1.0').replace('2_3_0', '2_3.0')
    test_path = f'{feat[:8]}test_{feat[8:]}.gz'.replace('2_1_0', '2_1.0').replace('2_3_0', '2_3.0')
    if  os.path.exists(f'../{feature_season}/3_third_valid/{train_path}'):
        continue
    if  os.path.exists(f'../{feature_season}/3_third_valid/{test_path}'):
        continue
    shutil.move(f'../{feature_season}/4_winner/{train_path}', f'../{feature_season}/3_third_valid/')
    shutil.move(f'../{feature_season}/4_winner/{test_path}', f'../{feature_season}/3_third_valid/')